In [ ]:
import pandas as pd
import time
import re
import numpy as np
import requests

In [ ]:
df = pd.read_excel("OCA Data.xlsx")
# df = pd.read_csv("real data lengkap.csv")
df

In [ ]:
# df = df.iloc[0:25].reset_index(drop=True)
# df

In [ ]:
# Konfigurasi API LLAMA Telkom AI
url = "[url]"
headers = {
    'accept': 'application/json',
    'x-api-key': "[api-key]",
    'Content-Type': 'application/json'
}

In [ ]:
def generate_question_answer(text):
    """
    Fungsi untuk mengidentifikasi satu pertanyaan dan jawaban yang paling relevan dari percakapan pelanggan menggunakan API LLAMA31.
    """
    prompt = f"""
    ROLE: 
    Kamu adalah Analis Customer Service Profesional yang bertugas menganalisis percakapan antara customer dan agent untuk mengekstrak pertanyaan dari customer serta jawaban yang diberikan oleh agent.
    
    INSTRUCTION: 
    Analisis dan ekstrak pertanyaan dari customer dan jawaban dari agent dari percakapan berikut '{text}'. Berdasarkan instruksi berikut:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak menanyakan data real-time, status pesanan, tiket, status email masuk, status gangguan aplikasi, atau akses ke sistem internal.
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer. Seperti meminta mengisi data diri, nomor telepon, email, alamat, dll. 
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut atau menunggu balasan dari pihak lain atau melakukan pengecekan atau meminta konfirmasi. (contoh: "Silakan tunggu/Mohon tunggu", "Kami akan melakukan pengecekan", "Mohon konfirmasi")
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV) atau nomor pesanan (PO) atau nomor pelanggan (CIF) atau LOP atau nomor lainnya.
    11. Pertanyaan tidak menanyakan hal mengenai status diri sendiri (seperti: "Bagaimana saya bisa membayar tagihan saya?", "Apakah saya sudah melakukan pembayaran tagihan saya?")
    12. Jawaban tidak meminta capture atau screenshoot atau meminta informasi yang hanya bisa dilihat di layar customer
    13. Jawaban tidak memberikan pernyataan bahwa agent telah melakukan tindakan tertentu atau telah menerima informasi tertentu dari customer. (contoh: "Saya telah mengirimkan email kepada Anda", "Kami telah menerima perimintaan Anda ...", "Kami telah membantu Anda ...")
    
    SCENARIO:
    Kamu akan menganalisis percakapan antara customer dan agent, lalu ekstrak pertanyaan dan jawaban dari percakapan yang sesuai dengan instruksi di atas.
    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas dan tidak sesuai instruksi, berikan "No data".
    Jika agent menyebutkan Nama orang atau seperti "Pak/Bu" pada jawaban, HAPUS nama atau ganti dengan "Anda" lalu sesuaikan kembali jawaban.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    EXTRACT:
    Pertanyaan: [Tuliskan hanya SATU pertanyaan dari customer yang sesuai instruksi dalam bentuk kalimat tanya yang jelas]
    Jawaban: [Tuliskan hanya SATU jawaban dari agent yang sesuai instruksi secara singkat, padat, jelas, dan mudah dipahami]
    """
    
    payload = {
        "message": [{"role": "user", "content": prompt}]
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=300)
        response_data = response.json().get("result", "")
        print(f"Output: {response_data}\n")
        
        if "Pertanyaan:" in response_data and "Jawaban:" in response_data:
            parts = response_data.split("Jawaban:")
            pertanyaan = parts[0].replace("Pertanyaan:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        else:
            pertanyaan, jawaban = "No data", "No data"
        
        return pertanyaan, jawaban
    
    except Exception as e:
        print(f"Error generating question-answer: {e}")
        return "Error", "Error"

def process_conversations_llama31(df, batch_size=1000, sleep_time=60):
    """
    Fungsi untuk memproses percakapan dan mengekstrak 'Pertanyaan' dan 'Jawaban' menggunakan LLAMA31.
    """
    total_rows = len(df)
    
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.loc[i, "Conversation"]
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            pertanyaan, jawaban = generate_question_answer(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
        
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df

In [ ]:
%%time
df = process_conversations_llama31(df)
df

In [ ]:
df.to_csv("oca_test1.csv", index=False)

In [ ]:
df

In [ ]:
# Filter the DataFrame for rows where 'Pertanyaan' or 'Jawaban' is "Error"
error_rows = df[(df['Pertanyaan'] == "Error") | (df['Jawaban'] == "Error")]
    
    # Display the filtered DataFrame
error_rows

In [ ]:
df = pd.read_csv("oca_test1.csv")
df

In [ ]:
def generate_question_answer(text):
    """
    Fungsi untuk mengidentifikasi satu pertanyaan dan jawaban yang paling relevan dari percakapan pelanggan menggunakan API LLAMA31.
    """
    prompt = f"""
    ROLE: 
    Kamu adalah Analis Customer Service Profesional yang bertugas menganalisis percakapan antara customer dan agent untuk mengekstrak pertanyaan dari customer serta jawaban yang diberikan oleh agent.
    
    INSTRUCTION: 
    Analisis dan ekstrak pertanyaan dari customer dan jawaban dari agent dari percakapan berikut '{text}'. Berdasarkan instruksi berikut:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak menanyakan data real-time, status pesanan, tiket, status email masuk, status gangguan aplikasi, atau akses ke sistem internal.
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer. Seperti meminta mengisi data diri, nomor telepon, email, alamat, dll. 
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut atau menunggu balasan dari pihak lain atau melakukan pengecekan atau meminta konfirmasi. (contoh: "Silakan tunggu/Mohon tunggu", "Kami akan melakukan pengecekan", "Mohon konfirmasi")
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV) atau nomor pesanan (PO) atau nomor pelanggan (CIF) atau LOP atau nomor lainnya.
    11. Pertanyaan tidak menanyakan hal mengenai status diri sendiri (seperti: "Bagaimana saya bisa membayar tagihan saya?", "Apakah saya sudah melakukan pembayaran tagihan saya?")
    12. Jawaban tidak meminta capture atau screenshoot atau meminta informasi yang hanya bisa dilihat di layar customer
    13. Jawaban tidak memberikan pernyataan bahwa agent telah melakukan tindakan tertentu atau telah menerima informasi tertentu dari customer. (contoh: "Saya telah mengirimkan email kepada Anda", "Kami telah menerima perimintaan Anda ...", "Kami telah membantu Anda ...")
    
    SCENARIO:
    Kamu akan menganalisis percakapan antara customer dan agent, lalu ekstrak pertanyaan dan jawaban dari percakapan yang sesuai dengan instruksi di atas.
    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas dan tidak sesuai instruksi, berikan "No data".
    Jika agent menyebutkan Nama orang atau seperti "Pak/Bu" pada jawaban, HAPUS nama atau ganti dengan "Anda" lalu sesuaikan kembali jawaban.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    EXTRACT:
    Pertanyaan: [Tuliskan hanya SATU pertanyaan dari customer yang sesuai instruksi dalam bentuk kalimat tanya yang jelas]
    Jawaban: [Tuliskan hanya SATU jawaban dari agent yang sesuai instruksi secara singkat, padat, jelas, dan mudah dipahami]
    """
    
    payload = {
        "message": [{"role": "user", "content": prompt}]
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=300)
        response_data = response.json().get("result", "")
        
        if "Pertanyaan:" in response_data and "Jawaban:" in response_data:
            parts = response_data.split("Jawaban:")
            pertanyaan = parts[0].replace("Pertanyaan:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        else:
            pertanyaan, jawaban = "No data", "No data"
        
        return pertanyaan, jawaban
    
    except Exception as e:
        print(f"Error generating question-answer: {e}")
        return "Error", "Error"


def process_conversations_llama31(df, batch_size=1000, sleep_time=60):
    """
    Fungsi untuk memproses percakapan dan mengekstrak 'Pertanyaan' dan 'Jawaban' menggunakan LLAMA31.
    """
    total_rows = len(df)
    
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.loc[i, "Conversation"]
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            pertanyaan, jawaban = generate_question_answer(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
        
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df


def reprocess_errors(df, batch_size=1000, sleep_time=60):
    """
    Fungsi untuk memproses ulang hanya baris yang memiliki Error atau No data.
    """
    # Filter baris yang perlu diproses ulang
    error_df = df[(df['Pertanyaan'].isin(["Error"])) | (df['Jawaban'].isin(["Error"]))]
    total_rows = len(error_df)

    if total_rows == 0:
        print("Tidak ada data yang perlu diproses ulang.")
        return df

    print(f"Memproses ulang {total_rows} baris yang Error...")

    for index in error_df.index:
        try:
            text = df.loc[index, "Conversation"]
            if pd.isna(text) or text.strip() == "":
                df.loc[index, 'Pertanyaan'] = "No data"
                df.loc[index, 'Jawaban'] = "No data"
                continue
            
            pertanyaan, jawaban = generate_question_answer(text)
            df.loc[index, 'Pertanyaan'] = pertanyaan
            df.loc[index, 'Jawaban'] = jawaban
        
        except Exception as e:
            print(f"[{index}]> Error: {e}")
            df.loc[index, 'Pertanyaan'] = "Error"
            df.loc[index, 'Jawaban'] = "Error"
        
        if (index + 1) % batch_size == 0:
            print(f"Processed {index + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    print("Proses ulang selesai.")
    return df


In [ ]:
%%time
df = reprocess_errors(df)

In [ ]:
df = pd.read_csv("mytens_test1.csv")
df

In [ ]:
def clean_data(df):
    # Fungsi untuk membersihkan pertanyaan
    def clean_question(text):
        if not isinstance(text, str):
            return text

        text = text.replace('*', '').strip()  # Menghilangkan karakter '*'
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]  # Mengambil paragraf yang tidak kosong
        if not paragraphs:
            return text

        # Jika ada pola seperti 'Pertanyaan 1:' atau 'Pertanyaan x:' ambil isi setelahnya
        match = re.search(r'Pertanyaan\s*\d+:\s*(.*)', text, re.DOTALL)
        if match:
            question_text = match.group(1).strip()
            # Jika pertanyaan terpotong atau berlanjut, hanya ambil kalimat pertama
            question_text = question_text.split('\n')[0].strip()
            return question_text

        # Jika tidak ada pola tersebut, ambil paragraf terakhir
        last_paragraph = paragraphs[-1]

        # Menghilangkan nomor di awal paragraf
        last_paragraph = re.sub(r'^\d+\.\s*', '', last_paragraph)
        return last_paragraph.strip()

    # Fungsi untuk membersihkan jawaban
    def clean_answer(text):
        if not isinstance(text, str):
            return text

        text = text.replace('*', '').strip()  # Menghilangkan karakter '*'
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]  # Mengambil paragraf yang tidak kosong
        if not paragraphs:
            return text

        # Jika jawaban berupa daftar prosedur bernomor, gabungkan menjadi satu paragraf
        if re.match(r'^\d+\.\s', paragraphs[0]):
            combined = ' '.join(paragraphs)
            return combined.strip()

        first_paragraph = paragraphs[0]  # Mengambil paragraf pertama jika bukan prosedur
        return first_paragraph.strip()

    # Menerapkan fungsi pembersihan ke kolom
    df['Pertanyaan'] = df['Pertanyaan'].apply(clean_question)
    df['Jawaban'] = df['Jawaban'].apply(clean_answer)

    return df

In [ ]:
clean_data(df)
df

In [ ]:
df.to_csv("test_mytens1_cleaned.csv", index=False)

In [ ]:
# Menghapus baris yang berisi "No Data"
df = df[(df['Pertanyaan'] != 'No data') & (df['Jawaban'] != 'No data')]

# Reset indeks
df = df.reset_index(drop=True)

# Menampilkan dataframe yang telah dibersihkan
df

In [ ]:
df.to_csv("OCA_llama_clean.csv", index=False)